In [9]:
import pandas as pd
import os
import re  # For extracting the year from sheet names

# Define file paths
excel_path = r"D:Downloads/datasets/mrt-3/ridership (2).xlsx"  # Update if needed
output_path = r"C:\Users\franj\notebooks\mrt\cleaned_ridership_data.csv"

# Read all sheets into a dictionary of DataFrames
sheets = pd.read_excel(excel_path, sheet_name=None)

# Inspect sheet names
print("Sheets found:", sheets.keys())

df_list = []
for sheet_name, df in sheets.items():
    # Extract year from sheet name (expects "DAILY YYYY" format)
    match = re.search(r"\d{4}", sheet_name)  # Finds a 4-digit year
    year = int(match.group()) if match else None  # Convert to integer
    
    if year is None:
        print(f"Warning: Could not extract year from sheet name '{sheet_name}'")
        continue  # Skip this sheet if year is missing
    
    df["Year"] = year  # Add a column to store the extracted year

    # Print first few rows for debugging
    print(f"\nFirst few rows of {sheet_name} (Year {year}):")
    print(df.head())

    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)

# Drop "Unnamed" columns (empty column headers)
df = df.loc[:, ~df.columns.str.contains("Unnamed")]

# Ensure first row isn't mistakenly treated as data instead of column names
if df.columns[0] != "Date":
    df.columns = df.iloc[0]  # Set first row as column names
    df = df[1:]  # Drop the now duplicated first row

# Convert ridership columns to numeric (handling errors like #DIV/0!)
for col in df.columns:
    if col not in ["Date", "Year"]:  # Skip non-ridership columns
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Check for any remaining non-numeric columns
print("\nNon-numeric columns:", df.select_dtypes(exclude=["number"]).columns.tolist())

# Remove rows where the railway was non-operational (i.e., all ridership values are NaN)
df = df.dropna(how="all", subset=df.columns[1:])  # Ignore 'Date' column in filtering

# Fill missing values with 0
df.fillna(0, inplace=True)

# Save cleaned dataset
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Ensure directory exists
df.to_csv(output_path, index=False)

print(f"\nCleaned dataset saved to: {output_path}")
print("Final dataset preview:")
print(df.head())

Sheets found: dict_keys(['DAILY 1999', 'DAILY 2000', 'DAILY 2001', 'DAILY 2002', 'DAILY 2003', 'DAILY 2004', 'DAILY 2005', 'DAILY 2006', 'DAILY 2007', 'DAILY 2008', 'DAILY 2009', 'DAILY 2010', 'DAILY 2011', 'DAILY 2012', 'DAILY 2013', 'DAILY 2014', 'DAILY 2015', 'DAILY 2016', 'DAILY 2017', 'DAILY 2018', 'DAILY 2019', 'DAILY 2020', 'DAILY 2021', 'DAILY 2022', 'DAILY 2023', 'DAILY 2024', 'DAILY 2025'])

First few rows of DAILY 1999 (Year 1999):
   Date  January  February  March  April  May  June  July  August  September  \
0     1      NaN       NaN    NaN    NaN  NaN   NaN   NaN     NaN        NaN   
1     2      NaN       NaN    NaN    NaN  NaN   NaN   NaN     NaN        NaN   
2     3      NaN       NaN    NaN    NaN  NaN   NaN   NaN     NaN        NaN   
3     4      NaN       NaN    NaN    NaN  NaN   NaN   NaN     NaN        NaN   
4     5      NaN       NaN    NaN    NaN  NaN   NaN   NaN     NaN        NaN   

   October  November  December  Year  
0      NaN       NaN       NaN  1